# Лабораторная работа 4, Степанов 6408, 1 вариант

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
device = '/device:GPU:0'

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [8]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [9]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the MNIST training set and periodically checks
    accuracy on the MNIST validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        i = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if i % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(i, epoch + 1,
                                             train_loss.result(),
                                             train_accuracy.result() * 100,
                                             val_loss.result(),
                                             val_accuracy.result() * 100))
                    i += 1

In [10]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2
print_every = 100

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.014145851135254, Accuracy: 6.25, Val Loss: 3.0509653091430664, Val Accuracy: 10.899999618530273
Iteration 100, Epoch 1, Loss: 2.2698535919189453, Accuracy: 28.929454803466797, Val Loss: 1.9187517166137695, Val Accuracy: 37.29999923706055
Iteration 200, Epoch 1, Loss: 2.1047921180725098, Accuracy: 32.291664123535156, Val Loss: 1.8691844940185547, Val Accuracy: 37.79999923706055
Iteration 300, Epoch 1, Loss: 2.0201940536499023, Accuracy: 34.08430099487305, Val Loss: 1.9366931915283203, Val Accuracy: 35.400001525878906
Iteration 400, Epoch 1, Loss: 1.9483729600906372, Accuracy: 35.81281280517578, Val Loss: 1.7539904117584229, Val Accuracy: 40.79999923706055
Iteration 500, Epoch 1, Loss: 1.9014551639556885, Accuracy: 36.860652923583984, Val Loss: 1.6435127258300781, Val Accuracy: 43.79999923706055
Iteration 600, Epoch 1, Loss: 1.8710941076278687, Accuracy: 37.741783142089844, Val Loss: 1.6900136470794678, Val Accuracy: 42.69999694824219
Iteration 700, Epoch 1,

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [11]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.908964157104492, Accuracy: 3.125, Val Loss: 4.109304904937744, Val Accuracy: 9.300000190734863
Iteration 100, Epoch 1, Loss: 1.9928560256958008, Accuracy: 29.702970504760742, Val Loss: 1.7378178834915161, Val Accuracy: 39.39999771118164
Iteration 200, Epoch 1, Loss: 1.8243969678878784, Accuracy: 36.03855514526367, Val Loss: 1.5481557846069336, Val Accuracy: 47.20000076293945
Iteration 300, Epoch 1, Loss: 1.723084568977356, Accuracy: 39.48297119140625, Val Loss: 1.4457783699035645, Val Accuracy: 49.0
Iteration 400, Epoch 1, Loss: 1.6429522037506104, Accuracy: 41.992671966552734, Val Loss: 1.3848601579666138, Val Accuracy: 49.39999771118164
Iteration 500, Epoch 1, Loss: 1.5847866535186768, Accuracy: 43.96519470214844, Val Loss: 1.3341491222381592, Val Accuracy: 53.10000228881836
Iteration 600, Epoch 1, Loss: 1.5476893186569214, Accuracy: 45.15391159057617, Val Loss: 1.2773367166519165, Val Accuracy: 54.70000076293945
Iteration 700, Epoch 1, Loss: 1.514060854

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [12]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

C:\Users\_Asus_\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 3.21211576461792, Accuracy: 14.0625, Val Loss: 2.828990936279297, Val Accuracy: 11.5
Iteration 100, Epoch 1, Loss: 2.2600533962249756, Accuracy: 27.985767364501953, Val Loss: 1.9356732368469238, Val Accuracy: 36.70000076293945
Iteration 200, Epoch 1, Loss: 2.089318037033081, Accuracy: 32.08177947998047, Val Loss: 1.82965886592865, Val Accuracy: 39.79999923706055
Iteration 300, Epoch 1, Loss: 2.0083937644958496, Accuracy: 34.068729400634766, Val Loss: 1.904189109802246, Val Accuracy: 36.29999923706055
Iteration 400, Epoch 1, Loss: 1.9385706186294556, Accuracy: 35.921913146972656, Val Loss: 1.7473958730697632, Val Accuracy: 40.099998474121094
Iteration 500, Epoch 1, Loss: 1.8931779861450195, Accuracy: 36.913673400878906, Val Loss: 1.649998664855957, Val Accuracy: 42.79999923706055
Iteration 600, Epoch 1, Loss: 1.860473871231079, Accuracy: 37.83277893066406, Val Loss: 1.6989644765853882, Val Accuracy: 41.0
Iteration 700, Epoch 1, Loss: 1.8342509269714355, Accur

Альтернативный менее гибкий способ обучения:

In [13]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - loss: 2.0074 - sparse_categorical_accuracy: 0.3399 - val_loss: 1.7084 - val_sparse_categorical_accuracy: 0.4250
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 1.7269 - sparse_categorical_accuracy: 0.4208


[1.736594319343567, 0.412200003862381]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [14]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    channel_1, channel_2, num_classes = 32, 16, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Input(shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)
    
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.65860652923584, Accuracy: 7.8125, Val Loss: 2.7778186798095703, Val Accuracy: 8.399999618530273
Iteration 100, Epoch 1, Loss: 2.000427722930908, Accuracy: 29.4554443359375, Val Loss: 1.811445951461792, Val Accuracy: 35.400001525878906
Iteration 200, Epoch 1, Loss: 1.8889693021774292, Accuracy: 33.582088470458984, Val Loss: 1.705072283744812, Val Accuracy: 40.900001525878906
Iteration 300, Epoch 1, Loss: 1.8211898803710938, Accuracy: 35.86482620239258, Val Loss: 1.666601538658142, Val Accuracy: 42.0
Iteration 400, Epoch 1, Loss: 1.7637780904769897, Accuracy: 37.916927337646484, Val Loss: 1.615571141242981, Val Accuracy: 44.10000228881836
Iteration 500, Epoch 1, Loss: 1.7257392406463623, Accuracy: 39.156063079833984, Val Loss: 1.5544507503509521, Val Accuracy: 44.80000305175781
Iteration 600, Epoch 1, Loss: 1.6999002695083618, Accuracy: 40.19862747192383, Val Loss: 1.5307878255844116, Val Accuracy: 47.20000076293945
Iteration 700, Epoch 1, Loss: 1.6750714778

In [15]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 25s 32ms/step - loss: 1.7813 - sparse_categorical_accuracy: 0.3718 - val_loss: 1.3082 - val_sparse_categorical_accuracy: 0.5440
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 1.3230 - sparse_categorical_accuracy: 0.5339


[1.3333759307861328, 0.5271999835968018]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [16]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [17]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0640270709991455, Accuracy: 4.6875, Val Loss: 2.8922085762023926, Val Accuracy: 12.800000190734863
Iteration 100, Epoch 1, Loss: 2.218022346496582, Accuracy: 29.11509895324707, Val Loss: 1.9195631742477417, Val Accuracy: 37.29999923706055
Iteration 200, Epoch 1, Loss: 2.0641775131225586, Accuracy: 32.781402587890625, Val Loss: 1.8192840814590454, Val Accuracy: 40.400001525878906
Iteration 300, Epoch 1, Loss: 1.9902102947235107, Accuracy: 34.55668640136719, Val Loss: 1.8994801044464111, Val Accuracy: 36.29999923706055
Iteration 400, Epoch 1, Loss: 1.9261804819107056, Accuracy: 36.167396545410156, Val Loss: 1.7342908382415771, Val Accuracy: 43.39999771118164
Iteration 500, Epoch 1, Loss: 1.8831270933151245, Accuracy: 37.30352020263672, Val Loss: 1.6530380249023438, Val Accuracy: 43.20000076293945
Iteration 600, Epoch 1, Loss: 1.8537262678146362, Accuracy: 38.22015380859375, Val Loss: 1.6764204502105713, Val Accuracy: 42.89999771118164
Iteration 700, Epoch 1,

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [22]:
class CustomConvNet_with_pool(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet_with_pool, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channels, num_classes = [32, 64], [100, 10]
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(channels[0], (5, 5), (3, 3), padding='same', kernel_initializer=initializer, input_shape=(32, 32, 3))
        self.conv2 = tf.keras.layers.Conv2D(channels[1], (3, 3), (3, 3), padding='same', kernel_initializer=initializer)
        self.pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        
        # Слой для развертывания
        self.fl = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes[0], activation='relu', kernel_initializer=initializer)  
        self.fc2 = tf.keras.layers.Dense(num_classes[1], activation='softmax', kernel_initializer=initializer)             

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)        
        x = self.conv2(x)
        x = self.pool(x)

        x = self.fl(x)
        x = self.fc1(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 6

model = CustomConvNet_with_pool()

def model_init_fn():
    return CustomConvNet_with_pool()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 4.795870780944824, Accuracy: 12.5, Val Loss: 4.7662811279296875, Val Accuracy: 9.399999618530273
Iteration 700, Epoch 1, Loss: 1.762420654296875, Accuracy: 37.3618049621582, Val Loss: 1.5523420572280884, Val Accuracy: 44.80000305175781
Iteration 1400, Epoch 2, Loss: 1.4625232219696045, Accuracy: 47.71653366088867, Val Loss: 1.4199520349502563, Val Accuracy: 49.900001525878906
Iteration 2100, Epoch 3, Loss: 1.3498191833496094, Accuracy: 51.688819885253906, Val Loss: 1.350987434387207, Val Accuracy: 51.79999923706055
Iteration 2800, Epoch 4, Loss: 1.2698688507080078, Accuracy: 54.41103744506836, Val Loss: 1.2900201082229614, Val Accuracy: 53.60000228881836
Iteration 3500, Epoch 5, Loss: 1.2121399641036987, Accuracy: 56.51101303100586, Val Loss: 1.273508906364441, Val Accuracy: 55.900001525878906
Iteration 4200, Epoch 6, Loss: 1.1627172231674194, Accuracy: 58.490570068359375, Val Loss: 1.2507306337356567, Val Accuracy: 54.900001525878906


In [24]:
def model_init_fn():
    return CustomConvNet_with_pool()

# Let's try Adam optimizer and another architecture
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 4.380430221557617, Accuracy: 14.0625, Val Loss: 4.403530120849609, Val Accuracy: 11.5
Iteration 700, Epoch 1, Loss: 2.4320335388183594, Accuracy: 18.774518966674805, Val Loss: 2.1207692623138428, Val Accuracy: 25.400001525878906
Iteration 1400, Epoch 2, Loss: 2.061908006668091, Accuracy: 26.96604347229004, Val Loss: 1.9873653650283813, Val Accuracy: 29.30000114440918
Iteration 2100, Epoch 3, Loss: 1.9552184343338013, Accuracy: 30.308107376098633, Val Loss: 1.9204496145248413, Val Accuracy: 32.20000076293945
Iteration 2800, Epoch 4, Loss: 1.891158938407898, Accuracy: 32.52671432495117, Val Loss: 1.8738330602645874, Val Accuracy: 33.70000076293945
Iteration 3500, Epoch 5, Loss: 1.8468022346496582, Accuracy: 34.01387405395508, Val Loss: 1.8388056755065918, Val Accuracy: 34.5
Iteration 4200, Epoch 6, Loss: 1.8137778043746948, Accuracy: 35.381568908691406, Val Loss: 1.8050193786621094, Val Accuracy: 36.29999923706055


In [25]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channels, num_classes = [64, 32], 10
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(channels[0], (5, 5), (1, 1), padding='same', kernel_initializer=initializer, input_shape=(32, 32, 3))
        self.conv2 = tf.keras.layers.Conv2D(channels[1], (3, 3), (1, 1), padding='same', kernel_initializer=initializer)

        # Нормализация
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()

        # Определение функции активации        
        self.relu1 = tf.keras.layers.ReLU()
        self.relu2 = tf.keras.layers.ReLU()
        
        self.dp1 = tf.keras.layers.Dropout(0.2)
        self.dp2 = tf.keras.layers.Dropout(0.15)

        # Слой для развертывания
        self.fl = tf.keras.layers.Flatten()       
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)             

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)        
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dp1(x, training=training)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dp2(x, training=training)

        x = self.fl(x)
        x = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 6

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.8873910903930664, Accuracy: 17.1875, Val Loss: 10.52589225769043, Val Accuracy: 8.5
Iteration 700, Epoch 1, Loss: 2.0052242279052734, Accuracy: 45.8095588684082, Val Loss: 1.2658838033676147, Val Accuracy: 56.099998474121094
Iteration 1400, Epoch 2, Loss: 1.1091418266296387, Accuracy: 61.46407699584961, Val Loss: 1.2121129035949707, Val Accuracy: 59.60000228881836
Iteration 2100, Epoch 3, Loss: 0.9158633947372437, Accuracy: 68.23099517822266, Val Loss: 1.0578547716140747, Val Accuracy: 63.80000305175781
Iteration 2800, Epoch 4, Loss: 0.8230756521224976, Accuracy: 71.33760070800781, Val Loss: 1.1089565753936768, Val Accuracy: 62.80000305175781
Iteration 3500, Epoch 5, Loss: 0.7525578737258911, Accuracy: 74.06321716308594, Val Loss: 1.044313669204712, Val Accuracy: 65.20000457763672
Iteration 4200, Epoch 6, Loss: 0.6871738433837891, Accuracy: 75.9560317993164, Val Loss: 1.043519377708435, Val Accuracy: 65.80000305175781


Опишите все эксперименты, результаты. Сделайте выводы.

1. По результатам эксперемнтов было полученио оптимальное количество эпох для обучения: **6**, после которого улучшений не наблюдается.
2. Среди оптимизаторов SGD и Adam, лучше себя показал `Adam`, будучи протестированы на одинаковых настройках.
3. Были исследованы несколько архитектур сетей, состоящих из:
   - a. сверточных слоев, нормализации, функций активации, слоев *dropout*, слоя для развертывания и полносвязного слоя;
   - b. сверточных слоев, максимального пулинга для первых двух слоев, слоя для развертывания и полносвязных слоев.
4. В дополнение к пункту выше стоит отметить, что добавление батч нормализации, дропаутов и дополнительного слоя значительно улучшили точность почти на $11$%. Дропауты оказали наибольшее влияние на результат, около $6$%.
5. Функция активации *sigmoid* показала себя хуже всех, гораздо лучшие результаты получены при использовании функции *ReLU*.
6. Для проведения различных эксперементов необходимо большое количество времени.
7. По итогу на определенных параметрах была достигнута точность $76$% на тестовой выборке и $66$% на валидационной.